In [414]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta 

from src.data_prep import *

%matplotlib inline

In [443]:
data = pd.read_pickle('../junyi_df.pickle')

In [444]:
#Dropping Un-needed Columns
data = drop_columns(data)

In [445]:
#Changing Bools to Int
data = change_bool(data)


In [454]:
#Convert Unix datetime to DateTime
convert_dates(data, 'time_done')

In [455]:
#Choosing Date Range
data = select_date(data, 'time_done', '1/1/2014', '9/30/2014')


In [179]:
data.head()

,user_id,topic_mode,suggested,review_mode,time_done,time_taken,correct,count_attempts,hint_used,count_hints,earned_proficiency,points_earned
5,135350,0,1,0,2014-02-19 08:46:32.220080,9,1,1,0,0,0,225
6,126493,0,0,0,2014-03-21 01:56:56.571340,19,0,2,0,0,0,0
8,91712,0,1,0,2014-03-26 04:41:24.278610,4,1,1,0,0,0,195
9,13719,0,0,0,2014-03-06 09:30:38.458960,2,1,1,0,0,0,5
43,41750,1,1,0,2014-03-11 16:35:37.248730,15,1,1,0,0,0,225


In [456]:
#Creating User Table with sums of data points
users = data.groupby('user_id').sum()

In [457]:
user_table = pd.DataFrame(users)

In [458]:
#Creating a time variable rng
user_time = data.groupby('user_id')
rng = (user_time['time_done'].max() - user_time['time_done'].min()).dt.days

In [459]:
#Cutting Table to include only users with 10 or more days active
user_table = user_table[(rng>9)]

In [357]:
#Creating Squared Hints, Correct, Review_Mode(Unused in final )
# user_table['count_hints_sq'] = user_table['count_hints'] **2 
# user_table['count_attempts_sq'] = user_table['count_attempts'] **2 
# user_table['review_mode_sq'] = user_table['review_mode'] **2
# user_table['correct_sq'] = user_table['correct'] **2

In [460]:
#Creating a More than 10 activities columns
total_usage = data['user_id'].value_counts().sort_index()
user_table['total_usage'] = total_usage


#Unused in final table
#user_table['more_than_100_activities'] = (total_usage > 100).astype(int)
#user_table['more_than_1000_activities'] = (total_usage > 1000).astype(int)
#user_table['more_than_ten_activities'] = (total_usage > 10).astype(int)

In [186]:
#Creating a More than X day columns

In [385]:
user_time = data.groupby('user_id')
rng = (user_time['time_done'].max() - user_time['time_done'].min()).dt.days

In [461]:
#The columns below did not end up in the final table
#user_table['active_more_than_1_day'] = (rng > 0).astype(int)
#user_table['active_at_least_5_days'] = (rng >= 4).astype(int)
#user_table['active_at_least_10_days'] = (rng >= 9).astype(int)

active_from_last= (data['time_done'].max() - (user_time['time_done'].max())).dt.days
user_table['active_from_last'] = active_from_last

#active_from_last_sq = active_from_last ** 2
#user_table['active_from_last_sq'] = active_from_last_sq

In [408]:
#Creating a 7 day streak columns



In [462]:
one_before = ((data['time_done'].max())- timedelta(1))
two_before = ((data['time_done'].max())- timedelta(2))
three_before = ((data['time_done'].max())- timedelta(3))
four_before = ((data['time_done'].max())- timedelta(4)) 
five_before = ((data['time_done'].max())- timedelta(5))
six_before =((data['time_done'].max())- timedelta(6))
seven_before =((data['time_done'].max())- timedelta(7))

In [463]:
user_table['one_before'] = ((one_before - (user_time['time_done'].max())).dt.days == 0).astype(int)

In [464]:
user_table['two_before'] = ((two_before - (user_time['time_done'].max())).dt.days == 0).astype(int)
user_table['three_before'] = ((three_before - (user_time['time_done'].max())).dt.days == 0).astype(int)
user_table['four_before'] = ((four_before - (user_time['time_done'].max())).dt.days == 0).astype(int)
user_table['five_before'] = ((five_before - (user_time['time_done'].max())).dt.days == 0).astype(int)
user_table['six_before'] = ((six_before - (user_time['time_done'].max())).dt.days == 0).astype(int)
user_table['seven_before'] = ((seven_before - (user_time['time_done'].max())).dt.days == 0).astype(int)

In [194]:
#list(user_table.index)

In [ ]:
# Creating Target Column 
target = create_target('../junyi_df.pickle', '10/1/2014', '10/31/2014', user_table)


In [205]:
target

{4: 0,
 16: 0,
 26: 0,
 29: 0,
 35: 0,
 36: 0,
 39: 0,
 46: 0,
 47: 0,
 50: 0,
 52: 0,
 53: 0,
 70: 0,
 71: 0,
 73: 0,
 78: 1,
 83: 0,
 85: 0,
 89: 0,
 92: 0,
 93: 0,
 100: 0,
 101: 0,
 105: 0,
 109: 0,
 124: 0,
 130: 0,
 147: 0,
 155: 0,
 161: 0,
 167: 1,
 174: 0,
 176: 0,
 202: 0,
 208: 0,
 209: 1,
 213: 0,
 221: 0,
 229: 0,
 239: 0,
 248: 0,
 260: 0,
 263: 0,
 264: 0,
 267: 0,
 268: 0,
 274: 0,
 277: 0,
 281: 0,
 286: 0,
 289: 0,
 290: 0,
 297: 0,
 319: 0,
 324: 0,
 327: 0,
 332: 0,
 347: 0,
 350: 0,
 356: 1,
 358: 0,
 365: 0,
 380: 0,
 382: 0,
 384: 0,
 397: 0,
 402: 0,
 404: 0,
 405: 0,
 410: 0,
 414: 0,
 418: 0,
 420: 1,
 424: 0,
 425: 0,
 434: 0,
 435: 0,
 436: 0,
 437: 1,
 450: 0,
 453: 0,
 466: 0,
 467: 0,
 488: 0,
 493: 0,
 495: 0,
 513: 0,
 516: 0,
 517: 0,
 528: 0,
 533: 0,
 536: 0,
 538: 0,
 542: 0,
 560: 0,
 583: 0,
 584: 0,
 589: 0,
 600: 1,
 602: 0,
 607: 0,
 608: 0,
 611: 0,
 633: 0,
 650: 0,
 666: 0,
 667: 0,
 675: 0,
 676: 0,
 681: 0,
 685: 0,
 689: 0,
 702: 1,
 718:

In [424]:
#Dropping if colums if needed before adding target and saving
#user_table.drop(columns=['target'], inplace=True)


In [423]:
#user_table.drop(columns=['active_at_least_10_days', 'active_at_least_5_days', 'active_more_than_1_day'], inplace=True)

In [425]:
user_table['target'] = target.values()

In [200]:
#user_table.drop(columns=['active_more_than_1', 'active_at_least_5', 'active_at_least_10'], inplace=True)

In [426]:
user_table.head(5)

,topic_mode,suggested,review_mode,time_taken,correct,count_attempts,hint_used,count_hints,earned_proficiency,points_earned,...,sum_attempts,one_before,two_before,three_before,four_before,five_before,six_before,seven_before,total_usage,target
user_id,,,,,,,,,,,,,,,,,,,,,
52,995,417,128,27337,2762,3487,32,104,187,257158,...,3487,0,1,0,0,0,0,0,3038,0
85,65,63,0,5049,123,255,41,169,8,15682,...,255,0,0,0,0,0,0,0,184,0
100,32,0,0,2113,48,123,14,60,0,7325,...,123,0,0,0,0,0,0,0,73,1
101,213,404,31,9445,809,1023,3,5,65,104617,...,1023,0,0,0,0,0,0,0,888,0
109,1,3,0,877,90,153,1,4,4,4310,...,153,0,0,0,0,0,1,0,105,0


In [427]:
user_table.columns

Index(['topic_mode', 'suggested', 'review_mode', 'time_taken', 'correct',
       'count_attempts', 'hint_used', 'count_hints', 'earned_proficiency',
       'points_earned', 'more_than_ten_activities', 'more_than_100_activities',
       'more_than_1000_activities', 'active_from_last', 'sum_attempts',
       'one_before', 'two_before', 'three_before', 'four_before',
       'five_before', 'six_before', 'seven_before', 'total_usage', 'target'],
      dtype='object')

In [428]:
#Save table
# Make sure to change version

user_table.to_csv('../users_v3.csv')

In [ ]:
for col in list(user_table.columns.values): 
    fig, ax = plt.subplots()
    ax.scatter(user_table['target'], user_table[col])
    ax.set_title(col)


In [402]:
list(user_table.columns.values)

['topic_mode',
 'suggested',
 'review_mode',
 'time_taken',
 'correct',
 'count_attempts',
 'hint_used',
 'count_hints',
 'earned_proficiency',
 'points_earned',
 'more_than_ten_activities',
 'more_than_100_activities',
 'more_than_1000_activities',
 'active_from_last',
 'one_before',
 'two_before',
 'three_before',
 'four_before',
 'five_before',
 'six_before',
 'seven_before',
 'target']

In [79]:
# #Unused feature engineering
# #y = user_indexed.loc[35, 'count_hints'][-10:].sum()

# for user_id in set(new_data.index.values):
# #     days_active[user_id] = get_active_weeks(user_id)

# def get_last_10_hints(user_id):
# #     try: 
#         return len(set(new_data.loc[user_id, 'week']))
#     except:
# #         return 0

In [ ]:
#Creating a More than 1 day co